**Цели:**
1. Научиться выполнять HTTP-запросы с использованием библиотеки Requests.
2. Научиться сохранять данные из датафрейма в файлы xlsx и csv.
3. Научиться работать c Git и сохранять результаты своей работы в GitLab.


Продолжим работу с iTunes Search API
Документацию к iTunes Search API можно [найти здесь](https://developer.apple.com/library/archive/documentation/AudioVideo/Conceptual/iTuneSearchAPI/Searching.html#//apple_ref/doc/uid/TP40017632-CH5-SW1)

**Задача 1**

Импортируйте библиотеки, необходимые для дальнейшей работы

In [1]:
from socket import timeout
import requests
import json
import pandas as pd
import openpyxl

**Задача 2**

1. Выполните GET-запрос https://itunes.apple.com/lookup?id=518462 и распечатайте ответ в формате json.

In [3]:
response = requests.get('https://itunes.apple.com/lookup?id=518462', )
print(response.json())

{'resultCount': 1, 'results': [{'wrapperType': 'artist', 'artistType': 'Artist', 'artistName': 'Louis Armstrong', 'artistLinkUrl': 'https://music.apple.com/us/artist/louis-armstrong/518462?uo=4', 'artistId': 518462, 'amgArtistId': 52456, 'primaryGenreName': 'Jazz', 'primaryGenreId': 11}]}


2. Распечатайте код ответа.

In [4]:
print(response.status_code)

200


3. Распечатайте строку запроса.

    _Подсказка:_ строку запроса можно распечатать с помощью метода `url`.

In [5]:
print(response.url)

https://itunes.apple.com/lookup?id=518462


4. Сохраните в переменную `amg_artist_id` значение `amgArtistId` полученного ответа и распечатайте его.

In [6]:
amg_artist_id = response.json().get('results')[0].get('amgArtistId')
print(amg_artist_id)

52456


***Задача 4***

1. Выполните POST-запрос к к ресурсу `lookup`, чтобы найти первые 100 записей, содержащих альбомы для артиста с `amgArtistId`, равного  полученному ранее `amg_artist_id`. Укажите время ожидания запроса 60 секунд и распечатайте ответ в формате json.


In [7]:
albums = requests.post('https://itunes.apple.com/lookup', data = {'amgArtistId': amg_artist_id, 'entity' : 'album', 'limit' : 100}, timeout = 60)
print(albums.json())

{'resultCount': 101, 'results': [{'wrapperType': 'artist', 'artistType': 'Artist', 'artistName': 'Louis Armstrong', 'artistLinkUrl': 'https://music.apple.com/us/artist/louis-armstrong/518462?uo=4', 'artistId': 518462, 'amgArtistId': 52456, 'primaryGenreName': 'Jazz', 'primaryGenreId': 11}, {'wrapperType': 'collection', 'collectionType': 'Album', 'artistId': 73568, 'collectionId': 1445666702, 'amgArtistId': 6503, 'artistName': 'Ella Fitzgerald & Louis Armstrong', 'collectionName': 'Ella & Louis Christmas', 'collectionCensoredName': 'Ella & Louis Christmas', 'artistViewUrl': 'https://music.apple.com/us/artist/ella-fitzgerald/73568?uo=4', 'collectionViewUrl': 'https://music.apple.com/us/album/ella-louis-christmas/1445666702?uo=4', 'artworkUrl60': 'https://is1-ssl.mzstatic.com/image/thumb/Music115/v4/16/a2/75/16a27533-9fc4-843b-c8d3-d81a377a2395/16UMGIM78399.rgb.jpg/60x60bb.jpg', 'artworkUrl100': 'https://is1-ssl.mzstatic.com/image/thumb/Music115/v4/16/a2/75/16a27533-9fc4-843b-c8d3-d81a377

2. Выведите количество записей в ответе.

In [8]:
print(albums.json().get('resultCount'))

101


3. Посмотрите внимательно на список с записями и обратите внимание, что мы запрашивали 100 альбомов, то есть на одну запись меньше, чем нам показал счётчик.
**Ответьте, в чём причина: 101 запись так как 1- это строка с инфо об артисте, остальные 100- альбомы

***Задача 5***

1. С учётом ответа на предыдущий вопрос разделите список на два списка:
- `data1` — альбомы с 1 по 30,
- `data2` — оставшиеся альбомы.

    Распечатайте количество элементов в каждом списке.

In [9]:
data1 = albums.json().get('results')[1:31]
data2 = albums.json().get('results')[31:]
print(len(data1))
print(len(data2))

30
70


2. Создайте датафрейм `df1` на основе списка `data1` и распечатайте его, а затем запишите его на лист `albums` в файл `30_albums.xlsx` в директории проекта `api/practice/data`. Не забудьте перед этим создать нужную директорию.


In [10]:
df1 = pd.DataFrame(data1)
print(df1)

   wrapperType collectionType  artistId  collectionId  amgArtistId  \
0   collection          Album     73568    1445666702         6503   
1   collection          Album    518462    1434914801        52456   
2   collection          Album    518462     193896088        52456   
3   collection          Album    518462     201274363        52456   
4   collection          Album    518462    1644157235        52456   
5   collection          Album     73568    1431066643         6503   
6   collection          Album    518462    1425235202        52456   
7   collection          Album     73568    1469584174         6503   
8   collection          Album    518462    1440787250        52456   
9   collection          Album    518462     158691605        52456   
10  collection          Album     73568    1454734940         6503   
11  collection          Album    518462     516685592        52456   
12  collection          Album    518462    1425244065        52456   
13  collection      

Для удобства переиспользования путь к папке с данными удобно вынести в отдельную переменную. Оставьте текущий путь в `path` или пропишите полный путь от директории, в которой запущен Jupyter Notebook.

In [13]:
path = 'C:\\Users\\makar\\OneDrive\\Рабочий стол\\ДЗ_Продуктовый аналитик\\'

df1.to_excel(path + 'dz_makarova.xlsx', sheet_name='albums', index=False)

3. Создайте датафрейм `df2` на основе списка `data2` и распечатайте его, а затем запишите его в файл `70_albums.csv` в директории проекта `api/practice/data`, указав в качестве разделителя знак табуляции.

In [17]:
df2 = pd.DataFrame(data2)
print(df2)

df2.to_csv(path + '70_albums.csv', sep ='\t')

   wrapperType collectionType  artistId  collectionId  amgArtistId  \
0   collection          Album    518462     158673207      52456.0   
1   collection          Album    518462     193043791      52456.0   
2   collection          Album     73568    1425173241       6503.0   
3   collection          Album    518462     198154274      52456.0   
4   collection          Album    518462    1422000964      52456.0   
..         ...            ...       ...           ...          ...   
65  collection          Album    518462    1607165210      52456.0   
66  collection          Album    518462    1440749171      52456.0   
67  collection          Album    518462    1426743749      52456.0   
68  collection          Album    518462    1426601992      52456.0   
69  collection          Album    518462    1675405359      52456.0   

                              artistName  \
0                        Louis Armstrong   
1                        Louis Armstrong   
2      Ella Fitzgerald & Lo

**Задача 6**

Попробуйте сформировать запросы к [iTunes Search API](https://affiliate.itunes.apple.com/resources/documentation/itunes-store-web-service-search-api/) таким образом, чтобы получить статусы ответов, отличные от 200. Например, 400 и 404. Распечатайте коды статусов для каждого запроса.

_Подсказка:_ воспользуйтесь [шпаргалкой по кодам ответа HTTP](https://developer.mozilla.org/ru/docs/Web/HTTP/Status), чтобы понять, как следует изменить запрос, чтобы сервер вернул определённую ошибку в ответе.


In [21]:
import requests


base_url = "https://itunes.apple.com/search"
params_bad_request = {
    "term": "",
    "country": "US"
}
response_400 = requests.get(base_url, params=params_bad_request)
print(f"Status code for 400 error: {response_400.status_code}")

invalid_url = "https://itunes.apple.com/invalid_endpoint"
response_404 = requests.get(invalid_url)
print(f"Status code for 404 error: {response_404.status_code}")

Status code for 400 error: 200
Status code for 404 error: 404
